In [1]:
import pandas as pd
from pathlib import Path
import rasterio
import rasterio.mask
from rasterio.plot import show_hist
from rasterio.plot import show

import rasterstats
from descartes import PolygonPatch

from matplotlib import pyplot as plt
import geojson
from shapely.geometry import Point, mapping, shape
from fiona import collection
import fiona
import geopandas as gpd
import geoplot

Contributor: Susanne Brockmann

## Setting up data access

In [2]:
shape_file_folder = Path.cwd().parent.parent.joinpath('data', 'GADM', 'Senegal')
#print(shape_file_folder)
shape_file_name = "gadm36_SEN_2.shp"
shape_file_path = Path(shape_file_folder).joinpath(shape_file_name)
print(shape_file_path)

H:\Omdena-food\GPSDD-Food-Security-Senegal\data\GADM\Senegal\gadm36_SEN_2.shp


In [3]:
data_folder = Path.cwd().parent.parent.joinpath('data', )
#print(data_folder)
file_name = "spei_1901_2020_05x05.csv"
file_path = Path(data_folder).joinpath(file_name)
print(file_path)

H:\Omdena-food\GPSDD-Food-Security-Senegal\data\spei_1901_2020_05x05.csv


In [4]:
crop_file_name = "senegal_crops3.csv"
crop_file_path = Path(data_folder).joinpath(crop_file_name)
print(crop_file_path)

H:\Omdena-food\GPSDD-Food-Security-Senegal\data\senegal_crops3.csv


## Creating GeoPandasDataFrame

In [5]:
spei_df = pd.read_csv(file_path)

In [6]:
spei_df

,Unnamed: 0,lat,lon,time,spei_1mon,spei_3mon,spei_6mon,spei_12mon,spei_24mon,spei_48mon
0,0,12.75,-17.25,1901-01-16,NaN,NaN,NaN,NaN,NaN,NaN
1,1,12.75,-17.25,1901-02-15,NaN,NaN,NaN,NaN,NaN,NaN
2,2,12.75,-17.25,1901-03-16,NaN,NaN,NaN,NaN,NaN,NaN
3,3,12.75,-17.25,1901-04-16,NaN,NaN,NaN,NaN,NaN,NaN
4,4,12.75,-17.25,1901-05-16,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
135931,135931,16.25,-11.75,2018-08-16,-0.531367,-0.253433,-0.514749,-0.865471,-0.980369,-1.002338
135932,135932,16.25,-11.75,2018-09-16,-0.170468,-0.344083,-0.493491,-0.677415,-0.851664,-0.948143
135933,135933,16.25,-11.75,2018-10-16,0.549886,-0.393346,-0.293457,-0.527478,-0.797354,-0.885630
135934,135934,16.25,-11.75,2018-11-16,0.523999,0.045234,-0.228973,-0.480762,-0.768407,-0.871314


In [7]:
# create a geopandas dataframe and set the crs
spei_gpd = gpd.GeoDataFrame(spei_df, geometry=gpd.points_from_xy(spei_df.lon, spei_df.lat))
spei_gpd.crs ='EPSG:4326'
spei_gpd.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
spei_gpd

,Unnamed: 0,lat,lon,time,spei_1mon,spei_3mon,spei_6mon,spei_12mon,spei_24mon,spei_48mon,geometry
0,0,12.75,-17.25,1901-01-16,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000)
1,1,12.75,-17.25,1901-02-15,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000)
2,2,12.75,-17.25,1901-03-16,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000)
3,3,12.75,-17.25,1901-04-16,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000)
4,4,12.75,-17.25,1901-05-16,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000)
...,...,...,...,...,...,...,...,...,...,...,...
135931,135931,16.25,-11.75,2018-08-16,-0.531367,-0.253433,-0.514749,-0.865471,-0.980369,-1.002338,POINT (-11.75000 16.25000)
135932,135932,16.25,-11.75,2018-09-16,-0.170468,-0.344083,-0.493491,-0.677415,-0.851664,-0.948143,POINT (-11.75000 16.25000)
135933,135933,16.25,-11.75,2018-10-16,0.549886,-0.393346,-0.293457,-0.527478,-0.797354,-0.885630,POINT (-11.75000 16.25000)
135934,135934,16.25,-11.75,2018-11-16,0.523999,0.045234,-0.228973,-0.480762,-0.768407,-0.871314,POINT (-11.75000 16.25000)


## Reading the shapefiles

In [9]:
Senegal_2 = gpd.read_file(str(shape_file_path))
Senegal_2.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,SEN,Senegal,SEN.1_1,Dakar,None,SEN.1.1_1,Dakar,None,None,Département,Department,11,SN.DK.DK,"MULTIPOLYGON (((-17.46736 14.65097, -17.46736 ..."
1,SEN,Senegal,SEN.1_1,Dakar,None,SEN.1.2_1,Guédiawaye,None,None,Département,Department,14,SN.DK.GD,"POLYGON ((-17.40012 14.75731, -17.40051 14.757..."
2,SEN,Senegal,SEN.1_1,Dakar,None,SEN.1.3_1,Pikine,None,None,Département,Department,12,SN.DK.PK,"POLYGON ((-17.39897 14.73781, -17.39903 14.737..."
3,SEN,Senegal,SEN.1_1,Dakar,None,SEN.1.4_1,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632..."
4,SEN,Senegal,SEN.2_1,Diourbel,None,SEN.2.1_1,Bambey,None,None,Département,Department,31,SN.DB.BB,"POLYGON ((-16.37093 14.60763, -16.37140 14.607..."


## Joining the two datasets

In [33]:
spei_gpd_geo = gpd.sjoin(spei_gpd, Senegal_2[["NAME_1", "NAME_2", "geometry"]], how='left', op ='within')
spei_gpd_geo

,Unnamed: 0,lat,lon,time,spei_1mon,spei_3mon,spei_6mon,spei_12mon,spei_24mon,spei_48mon,geometry,index_right,NAME_1,NAME_2
0,0,12.75,-17.25,1901-01-16,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000),NaN,NaN,NaN
1,1,12.75,-17.25,1901-02-15,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000),NaN,NaN,NaN
2,2,12.75,-17.25,1901-03-16,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000),NaN,NaN,NaN
3,3,12.75,-17.25,1901-04-16,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000),NaN,NaN,NaN
4,4,12.75,-17.25,1901-05-16,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-17.25000 12.75000),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135931,135931,16.25,-11.75,2018-08-16,-0.531367,-0.253433,-0.514749,-0.865471,-0.980369,-1.002338,POINT (-11.75000 16.25000),NaN,NaN,NaN
135932,135932,16.25,-11.75,2018-09-16,-0.170468,-0.344083,-0.493491,-0.677415,-0.851664,-0.948143,POINT (-11.75000 16.25000),NaN,NaN,NaN
135933,135933,16.25,-11.75,2018-10-16,0.549886,-0.393346,-0.293457,-0.527478,-0.797354,-0.885630,POINT (-11.75000 16.25000),NaN,NaN,NaN
135934,135934,16.25,-11.75,2018-11-16,0.523999,0.045234,-0.228973,-0.480762,-0.768407,-0.871314,POINT (-11.75000 16.25000),NaN,NaN,NaN


In [34]:
spei_gpd_geo.NAME_2.unique()

array([nan, 'Bignona', 'Sédhiou', 'Goudomp', 'Kolda', 'Vélingara',
       'Tambacounda', 'Kédougou', 'Saraya', 'Bounkiling',
       'Médina Yoro Foula', 'Foundiougne', 'Nioro du Rip', 'Koupentoum',
       'Goudiry', 'Bakel', 'Mbour', 'Kaolack', 'Birkilane',
       'Malème Hodar', 'Koungheul', 'Rufisque', 'Thiès', 'Diourbel',
       'Mbacké', 'Linguère', 'Ranérou Ferlo', 'Kanel', 'Tivaouane',
       'Kébémer', 'Louga', 'Podor', 'Matam', 'Dagana'], dtype=object)

### Filtering out points that are not within Senegal

In [35]:
spei_gpd_geo = spei_gpd_geo[spei_gpd_geo['NAME_2'].notnull()].copy()
spei_gpd_geo

,Unnamed: 0,lat,lon,time,spei_1mon,spei_3mon,spei_6mon,spei_12mon,spei_24mon,spei_48mon,geometry,index_right,NAME_1,NAME_2
1416,1416,12.75,-16.75,1901-01-16,-0.465913,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),42.0,Ziguinchor,Bignona
1417,1417,12.75,-16.75,1901-02-15,-1.007143,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),42.0,Ziguinchor,Bignona
1418,1418,12.75,-16.75,1901-03-16,-0.035826,-0.710538,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),42.0,Ziguinchor,Bignona
1419,1419,12.75,-16.75,1901-04-16,-0.977676,-0.834841,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),42.0,Ziguinchor,Bignona
1420,1420,12.75,-16.75,1901-05-16,1.119939,0.731673,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),42.0,Ziguinchor,Bignona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128851,128851,16.25,-14.25,2018-08-16,-1.488464,-0.965211,-1.002726,-1.409793,-1.490871,-1.443985,POINT (-14.25000 16.25000),30.0,Saint-Louis,Podor
128852,128852,16.25,-14.25,2018-09-16,1.294965,-0.691331,-0.272361,-0.549097,-1.249870,-1.148024,POINT (-14.25000 16.25000),30.0,Saint-Louis,Podor
128853,128853,16.25,-14.25,2018-10-16,0.449137,-0.351602,-0.114808,-0.230830,-1.097047,-1.065837,POINT (-14.25000 16.25000),30.0,Saint-Louis,Podor
128854,128854,16.25,-14.25,2018-11-16,0.792088,1.155867,-0.166837,-0.201916,-1.082612,-1.047246,POINT (-14.25000 16.25000),30.0,Saint-Louis,Podor


### Cleaning up the dataframe by removing unwanted columns and reindexing

In [36]:
spei_gpd_geo.columns

Index(['Unnamed: 0', 'lat', 'lon', 'time', 'spei_1mon', 'spei_3mon',
       'spei_6mon', 'spei_12mon', 'spei_24mon', 'spei_48mon', 'geometry',
       'index_right', 'NAME_1', 'NAME_2'],
      dtype='object')

In [37]:
drop_cols = ['Unnamed: 0', 'index_right']

In [38]:
spei_gpd_geo.drop(columns = drop_cols, inplace = True, axis=1)

In [39]:
spei_gpd_geo

,lat,lon,time,spei_1mon,spei_3mon,spei_6mon,spei_12mon,spei_24mon,spei_48mon,geometry,NAME_1,NAME_2
1416,12.75,-16.75,1901-01-16,-0.465913,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
1417,12.75,-16.75,1901-02-15,-1.007143,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
1418,12.75,-16.75,1901-03-16,-0.035826,-0.710538,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
1419,12.75,-16.75,1901-04-16,-0.977676,-0.834841,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
1420,12.75,-16.75,1901-05-16,1.119939,0.731673,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
...,...,...,...,...,...,...,...,...,...,...,...,...
128851,16.25,-14.25,2018-08-16,-1.488464,-0.965211,-1.002726,-1.409793,-1.490871,-1.443985,POINT (-14.25000 16.25000),Saint-Louis,Podor
128852,16.25,-14.25,2018-09-16,1.294965,-0.691331,-0.272361,-0.549097,-1.249870,-1.148024,POINT (-14.25000 16.25000),Saint-Louis,Podor
128853,16.25,-14.25,2018-10-16,0.449137,-0.351602,-0.114808,-0.230830,-1.097047,-1.065837,POINT (-14.25000 16.25000),Saint-Louis,Podor
128854,16.25,-14.25,2018-11-16,0.792088,1.155867,-0.166837,-0.201916,-1.082612,-1.047246,POINT (-14.25000 16.25000),Saint-Louis,Podor


In [40]:
spei_gpd_geo.reset_index(drop=True)

,lat,lon,time,spei_1mon,spei_3mon,spei_6mon,spei_12mon,spei_24mon,spei_48mon,geometry,NAME_1,NAME_2
0,12.75,-16.75,1901-01-16,-0.465913,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
1,12.75,-16.75,1901-02-15,-1.007143,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
2,12.75,-16.75,1901-03-16,-0.035826,-0.710538,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
3,12.75,-16.75,1901-04-16,-0.977676,-0.834841,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
4,12.75,-16.75,1901-05-16,1.119939,0.731673,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona
...,...,...,...,...,...,...,...,...,...,...,...,...
94867,16.25,-14.25,2018-08-16,-1.488464,-0.965211,-1.002726,-1.409793,-1.490871,-1.443985,POINT (-14.25000 16.25000),Saint-Louis,Podor
94868,16.25,-14.25,2018-09-16,1.294965,-0.691331,-0.272361,-0.549097,-1.249870,-1.148024,POINT (-14.25000 16.25000),Saint-Louis,Podor
94869,16.25,-14.25,2018-10-16,0.449137,-0.351602,-0.114808,-0.230830,-1.097047,-1.065837,POINT (-14.25000 16.25000),Saint-Louis,Podor
94870,16.25,-14.25,2018-11-16,0.792088,1.155867,-0.166837,-0.201916,-1.082612,-1.047246,POINT (-14.25000 16.25000),Saint-Louis,Podor


In [41]:
col_names = {
    'NAME_1': 'level1',
    'NAME_2': 'level2'
}

spei_gpd_geo.rename(columns=col_names, inplace=True)

## Creating a year column

In [42]:
spei_gpd_geo['time'] = pd.to_datetime(spei_gpd_geo['time'])
spei_gpd_geo['year'] = spei_gpd_geo.time.dt.year
spei_gpd_geo

,lat,lon,time,spei_1mon,spei_3mon,spei_6mon,spei_12mon,spei_24mon,spei_48mon,geometry,level1,level2,year
1416,12.75,-16.75,1901-01-16,-0.465913,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona,1901
1417,12.75,-16.75,1901-02-15,-1.007143,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona,1901
1418,12.75,-16.75,1901-03-16,-0.035826,-0.710538,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona,1901
1419,12.75,-16.75,1901-04-16,-0.977676,-0.834841,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona,1901
1420,12.75,-16.75,1901-05-16,1.119939,0.731673,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),Ziguinchor,Bignona,1901
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128851,16.25,-14.25,2018-08-16,-1.488464,-0.965211,-1.002726,-1.409793,-1.490871,-1.443985,POINT (-14.25000 16.25000),Saint-Louis,Podor,2018
128852,16.25,-14.25,2018-09-16,1.294965,-0.691331,-0.272361,-0.549097,-1.249870,-1.148024,POINT (-14.25000 16.25000),Saint-Louis,Podor,2018
128853,16.25,-14.25,2018-10-16,0.449137,-0.351602,-0.114808,-0.230830,-1.097047,-1.065837,POINT (-14.25000 16.25000),Saint-Louis,Podor,2018
128854,16.25,-14.25,2018-11-16,0.792088,1.155867,-0.166837,-0.201916,-1.082612,-1.047246,POINT (-14.25000 16.25000),Saint-Louis,Podor,2018


In [43]:
spei_gpd_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 94872 entries, 1416 to 128855
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   lat         94872 non-null  float64       
 1   lon         94872 non-null  float64       
 2   time        94872 non-null  datetime64[ns]
 3   spei_1mon   94808 non-null  float64       
 4   spei_3mon   94730 non-null  float64       
 5   spei_6mon   94537 non-null  float64       
 6   spei_12mon  94135 non-null  float64       
 7   spei_24mon  93331 non-null  float64       
 8   spei_48mon  91723 non-null  float64       
 9   geometry    94872 non-null  geometry      
 10  level1      94872 non-null  object        
 11  level2      94872 non-null  object        
 12  year        94872 non-null  int64         
dtypes: datetime64[ns](1), float64(8), geometry(1), int64(1), object(2)
memory usage: 10.1+ MB


## Read crops data

In [19]:
crops_df = pd.read_csv(crop_file_path)
crops_df

,year,level1,level2,lat,lon,tempday,tempnight,ndvimin,ndvimean,ndvimax,...,prodCorn,prodSorghum,prodRice,prodGroundnuts,yieldSorghum,yieldMillet,yieldGroundnut,yieldCorn,yieldRice,popTotal
0,2009,Tambacounda,Bakel,14.904167,-12.458333,36.591301,22.035505,-0.140204,0.379052,0.658357,...,0.0,5148.0,669.0,1792.0,1826.0,823.0,768,1715.0,3000.0,140836
1,2010,Tambacounda,Bakel,14.904167,-12.458333,36.244129,22.187347,-0.145652,0.385272,0.670017,...,5608.0,3731.0,625.0,1702.0,1101.0,850.0,809,1732.0,3125.0,145478
2,2011,Tambacounda,Bakel,14.904167,-12.458333,37.102672,22.510456,-0.158083,0.350024,0.646398,...,862.0,3091.0,2360.0,365.0,554.0,639.0,809,463.0,5500.0,149453
3,2012,Tambacounda,Bakel,14.904167,-12.458333,35.741772,22.224788,-0.132280,0.382634,0.642796,...,7501.0,8549.0,4550.0,255.0,1866.0,981.0,948,1964.0,6500.0,153961
4,2013,Tambacounda,Bakel,14.904167,-12.458333,37.314345,22.785087,-0.118283,0.376694,0.667315,...,4331.0,3253.0,665.0,1144.0,720.0,918.0,856,1312.0,5500.0,490695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2012,Ziguinchor,Ziguinchor,12.583300,-16.266600,29.420593,19.601906,-0.189020,0.526170,0.837076,...,92.0,95.0,3559.0,836.0,816.0,894.0,1190,869.0,1810.0,345574
356,2013,Ziguinchor,Ziguinchor,12.583300,-16.266600,30.002963,20.181129,-0.174339,0.558435,0.868430,...,126.0,94.0,5510.0,1076.0,800.0,678.0,1006,734.0,2263.0,248265
357,2014,Ziguinchor,Ziguinchor,12.583300,-16.266600,29.916419,19.609041,-0.186600,0.537421,0.840563,...,216.0,120.0,3910.0,270.0,800.0,750.0,950,1200.0,1700.0,255855
358,2015,Ziguinchor,Ziguinchor,12.583300,-16.266600,29.349449,19.968154,-0.188780,0.548859,0.838746,...,191.0,223.0,11835.0,752.0,970.0,740.0,910,794.0,1973.0,263807


## Cleaning department names to prepare dataset fusion

In [73]:
crop_level2= set(crops_df.level2)
crop_level2

{'Bakel',
 'Bignona',
 'Birkilane',
 'Bounkiling',
 'Dagana',
 'Diourbel',
 'Foundiougne',
 'Goudiry',
 'Goudomp',
 'Guediawaye',
 'Kanel',
 'Kaolack',
 'Kolda',
 'Koungheul',
 'Koupentoum',
 'Kébémer',
 'Kédougou',
 'Linguère',
 'Louga',
 'Malème Hodar',
 'Matam',
 'Mbacké',
 'Mbour',
 'Médina Yoro Foula',
 'Nioro du Rip',
 'Podor',
 'Ranérou Ferlo',
 'Rufisque',
 'Salemata',
 'Saraya',
 'Sédhiou',
 'Tambacounda',
 'Thiès',
 'Tivaouane',
 'Vélingara'}

In [74]:
spei_level2= set(spei_gpd_geo.level2)
spei_level2

{'Bakel',
 'Bignona',
 'Birkilane',
 'Bounkiling',
 'Dagana',
 'Diourbel',
 'Foundiougne',
 'Goudiry',
 'Goudomp',
 'Kanel',
 'Kaolack',
 'Kolda',
 'Koungheul',
 'Koupentoum',
 'Kébémer',
 'Kédougou',
 'Linguère',
 'Louga',
 'Malème Hodar',
 'Matam',
 'Mbacké',
 'Mbour',
 'Médina Yoro Foula',
 'Nioro du Rip',
 'Podor',
 'Ranérou Ferlo',
 'Rufisque',
 'Saraya',
 'Sédhiou',
 'Tambacounda',
 'Thiès',
 'Tivaouane',
 'Vélingara'}

In [75]:
spei_level2.difference(crop_level2)

set()

In [76]:
crop_level2.difference(spei_level2)

{'Guediawaye', 'Salemata'}

In [77]:
spei_level2.symmetric_difference(crop_level2) # find the difference between the two names

{'Guediawaye', 'Salemata'}

In [71]:
departments_mapping ={
    "Birkelane": "Birkilane",
    "Kebemer": "Kébémer",
    "Kedougou": "Kédougou",
    "Koungueul": "Koungheul",
    "Koupentoump": "Koupentoum",
    "Linguere": "Linguère",
    "Maleme hodar": "Malème Hodar",
    "Mbacke": "Mbacké",
    "Medina yoro foula": "Médina Yoro Foula",
    "Nioro du rip": "Nioro du Rip",
    "Ranerou": "Ranérou Ferlo",
    "Sedhiou": "Sédhiou",
    "Thies": "Thiès",
    "Velingara": "Vélingara"    
}

In [72]:
crops_df.level2 = crops_df.level2.replace(departments_mapping)

In [79]:
spei_level2.symmetric_difference(crop_level2) # find the difference between the two names

{'Guediawaye', 'Salemata'}

IMPORTANT NOTE: Ville de Guédiawaye in the region of Dakar and Salemata in the region Kédougou are not in the grid dataset. Do you want to discard the data for these regions from your combined dataset?

## Joining the datasets

In [86]:
fusion_df = spei_gpd_geo.merge(crops_df, how="left", on=['level2', 'year']) 

In [87]:
fusion_df

,lat_x,lon_x,time,spei_1mon,spei_3mon,spei_6mon,spei_12mon,spei_24mon,spei_48mon,geometry,...,prodCorn,prodSorghum,prodRice,prodGroundnuts,yieldSorghum,yieldMillet,yieldGroundnut,yieldCorn,yieldRice,popTotal
0,12.75,-16.75,1901-01-16,-0.465913,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.75,-16.75,1901-02-15,-1.007143,NaN,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12.75,-16.75,1901-03-16,-0.035826,-0.710538,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.75,-16.75,1901-04-16,-0.977676,-0.834841,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12.75,-16.75,1901-05-16,1.119939,0.731673,NaN,NaN,NaN,NaN,POINT (-16.75000 12.75000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97555,16.25,-14.25,2018-08-16,-1.488464,-0.965211,-1.002726,-1.409793,-1.490871,-1.443985,POINT (-14.25000 16.25000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97556,16.25,-14.25,2018-09-16,1.294965,-0.691331,-0.272361,-0.549097,-1.249870,-1.148024,POINT (-14.25000 16.25000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97557,16.25,-14.25,2018-10-16,0.449137,-0.351602,-0.114808,-0.230830,-1.097047,-1.065837,POINT (-14.25000 16.25000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97558,16.25,-14.25,2018-11-16,0.792088,1.155867,-0.166837,-0.201916,-1.082612,-1.047246,POINT (-14.25000 16.25000),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
fusion_df.prodCorn.unique()

array([       nan, 1.2181e+04, 4.0060e+03, 2.6520e+03, 1.1182e+04,
       9.1070e+03, 9.7360e+03, 1.9437e+04, 1.9435e+04, 6.8850e+03,
       7.6850e+03, 5.0230e+03, 6.5210e+03, 3.4290e+03, 3.4750e+03,
       8.4800e+03, 9.2090e+03, 1.5176e+04, 6.0590e+03, 1.2454e+04,
       2.2261e+04, 1.3661e+04, 1.3721e+04, 2.6843e+04, 3.8387e+04,
       0.0000e+00, 1.0613e+04, 6.7460e+03, 1.2907e+04, 1.1483e+04,
       7.1220e+03, 1.6144e+04, 1.1427e+04, 2.9985e+04, 2.5850e+03,
       7.4200e+02, 2.2320e+03, 4.9920e+03, 1.0540e+04, 3.7286e+04,
       4.3164e+04, 5.5790e+03, 3.3060e+03, 4.6820e+03, 5.1270e+03,
       2.2970e+03, 4.7240e+03, 5.5890e+03, 4.5660e+03, 9.1340e+03,
       6.2510e+03, 5.1410e+03, 4.4840e+03, 9.8470e+03, 5.3190e+03,
       1.0581e+04, 8.6470e+03, 1.1300e+03, 2.0760e+03, 7.2000e+01,
       3.0100e+02, 1.1400e+02, 1.4550e+03, 7.7000e+01, 1.6570e+03,
       7.1000e+01, 2.7600e+03, 6.0000e+01, 3.4350e+03, 6.6000e+01,
       3.2000e+03, 1.3100e+02, 5.7400e+02, 3.0500e+02, 3.5900e